# Breakout Atari

In this environment, the observation is an **RGB image** of the screen, which is an array of shape **(210, 160, 3)** Each action is repeatedly performed for a duration of *k* frames, where *k* is uniformly sampled from {2, 3, 4}. 

This will be further explained in **explore the environment** section


### Game rules

Breakout begins with eight rows of bricks, with each two rows a different color. The color order from the bottom up is **yellow**, **green**, **orange** and **red**.   

Using a single ball, the player must knock down as many bricks as possible by using the walls and/or the paddle below to ricochet the ball against the bricks and eliminate them. If the player’s paddle misses the ball’s rebound, he or she will lose a turn. The player has three turns to try to clear two screens of bricks. 

Color to points:
- Yellow bricks earn one point each 
- Green bricks earn three points 
- Orange bricks earn five points 
- The top-level red bricks score seven points each

The paddle shrinks to one-half its size after the ball has broken through the red row and hit the upper wall. Ball speed increases at specific intervals: after four hits, after twelve hits, and after making contact with the orange and red rows.


<img src="https://raw.githubusercontent.com/Kyushik/Unity_ML_Agent/master/Images/Breakout.png" width="400px" height="500px" />


*Rules description taken from [wikipedia](https://en.wikipedia.org/wiki/Breakout_%28video_game%29)*


### Checkout the example below

In [19]:
from IPython.display import HTML

HTML("""
<div align="middle">
<video width="20%" controls>
      <source src="./res/original.mp4" type="video/mp4">
</video>
</div>""")

## 1. Install necessary packages/libs

In [ ]:
!pip install 'stable-baselines3[extra]'
!pip install tensorboard==1.15.0
!python -m atari_py.import_roms rars

**NOTE**:  ROMS are predownloaded and included in the repository.

You can also download them yourself from [Atari ROMS](http://www.atarimania.com/rom_collection_archive_atari_2600_roms.html)

## 2. Imports

In [1]:
import gym 
import os

from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env

## 3. Create & Explore the environment

In [2]:
def make_environment(env_name):
    return gym.make(env_name)
    
env = make_environment("Breakout-v0")

In [ ]:
print(f"Action space type -> {env.action_space}")
print(f"Action space sample -> {env.action_space.sample()}")

In [ ]:
env.unwrapped.get_action_meanings()

We can see that action can take 4 values, that correspond to the following actions:

- 0 -> No operation 
- 1 -> Fire
- 2 -> Move right 
- 3 -> Move left


In [4]:
print(f"Observation space type -> {env.observation_space}")
print(f"Observation space sample -> {env.observation_space.sample()}")

Observation space type -> Box([[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]], [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 25

In [ ]:
print(f"Observation space shape -> {env.observation_space.sample().shape}")

#### More information on all possible [space types](https://gym.openai.com/docs/#spaces)

<br><br>
Breakout **does not** have a reward limit as, for example, CartPole does. 

This means it can be trained indefinitely.

## 4. Random dummy agent

In [ ]:
EPISODES = 10

In [ ]:
for ep in range(0, EPISODES):
    state = env.reset()
    is_done = False
    score = 0 
    
    while not is_done:
        
        env.render()
        
        # take a sample from action space (random action)
        action = env.action_space.sample()
        state, reward, is_done, additional_info = env.step(action)
        score += reward
    print(f'Step -> {ep+1} | Score -> {score}')
env.close()

## 5. Vectorise Environment

Vectorizing allows us to train our model faster because we are essentially training multiple environments in parallel (4 in our case)

In [3]:
env = make_atari_env('Breakout-v0', n_envs=4, seed=0)

In [4]:
env = VecFrameStack(env, n_stack=4)

<img src="./res/vectorized_envs.png" width="200px" height="250px" />

*Call to **env.render()** after vectorization*

## 6. Train Model

#### There are different kinds of RL algorithms that can be used for training, each with its own tradeoffs


<img src="https://spinningup.openai.com/en/latest/_images/rl_algorithms_9_15.svg" width="800px" height="1000px" />

*Source: [spinning up OpenAI docs](https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html)*


### How do we know which RL algorithm to use?
Some algorithms work **only** on certain types of action spaces

<img src="./res/action_space_to_algo.png" width="400px" height="500px" />


<br><br>
From observations about the environment performed above, we can see that  
we have a **Discrete** action space, which means we can use **A2C** for the task at hand.


### Understanding training metrics

<div><img src="./res/training_metrics_a2c.png" width="400px" height="500px" /></div>

*Training metric example*

##### Explaining most important metrics:

- *ep_len_mean* -> how long does the episode last on average
- *ep_reward_mean* -> average reward per episode


In [7]:
log_path = os.path.join('breakout_training', 'logs')

#### Add training reward threshold callback

In [5]:
# Let's go with 20 points
REWARD_THRESHOLD = 20

from stable_baselines3.common.callbacks import (
    EvalCallback,
    StopTrainingOnRewardThreshold
)

best_model_save_path = os.path.join('breakout_training', 'trained_models')

stop_callback = StopTrainingOnRewardThreshold(reward_threshold=REWARD_THRESHOLD, 
                                             verbose=1)
evaluation_callback = EvalCallback(env,
                                  callback_on_new_best=stop_callback,
                                  eval_freq=6000,
                                  best_model_save_path=best_model_save_path,
                                  verbose=1)

In [8]:
# Use CnnPolicy since our observations are returned as images
model = A2C("CnnPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [9]:
model.learn(total_timesteps=1000000, callback=evaluation_callback)

Logging to breakout_training/logs/A2C_3


/Users/markiyanvalyavka/opt/anaconda3/envs/ai37/lib/python3.7/site-packages/stable_baselines3/common/callbacks.py:337: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x7fe570249c50> != <stable_baselines3.common.vec_env.vec_frame_stack.VecFrameStack object at 0x7fe56a893450>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 264      |
|    ep_rew_mean        | 1.26     |
| time/                 |          |
|    fps                | 128      |
|    iterations         | 100      |
|    time_elapsed       | 15       |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -1.39    |
|    explained_variance | 0.0141   |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.0396   |
|    value_loss         | 0.107    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 276      |
|    ep_rew_mean        | 1.48     |
| time/                 |          |
|    fps                | 153      |
|    iterations         | 200      |
|    time_elapsed       | 26       |
|    total_timesteps    | 4000     |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 351      |
|    ep_rew_mean        | 3.01     |
| time/                 |          |
|    fps                | 174      |
|    iterations         | 1400     |
|    time_elapsed       | 160      |
|    total_timesteps    | 28000    |
| train/                |          |
|    entropy_loss       | -1.1     |
|    explained_variance | 0.742    |
|    learning_rate      | 0.0007   |
|    n_updates          | 1399     |
|    policy_loss        | -0.093   |
|    value_loss         | 0.0945   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 361      |
|    ep_rew_mean        | 3.25     |
| time/                 |          |
|    fps                | 175      |
|    iterations         | 1500     |
|    time_elapsed       | 171      |
|    total_timesteps    | 30000    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 419      |
|    ep_rew_mean        | 4.65     |
| time/                 |          |
|    fps                | 174      |
|    iterations         | 2700     |
|    time_elapsed       | 309      |
|    total_timesteps    | 54000    |
| train/                |          |
|    entropy_loss       | -0.23    |
|    explained_variance | 0.635    |
|    learning_rate      | 0.0007   |
|    n_updates          | 2699     |
|    policy_loss        | 0.198    |
|    value_loss         | 0.331    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 430      |
|    ep_rew_mean        | 4.9      |
| time/                 |          |
|    fps                | 175      |
|    iterations         | 2800     |
|    time_elapsed       | 319      |
|    total_timesteps    | 56000    |
| train/                |          |
|

...

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 799      |
|    ep_rew_mean        | 14       |
| time/                 |          |
|    fps                | 175      |
|    iterations         | 35000    |
|    time_elapsed       | 3994     |
|    total_timesteps    | 700000   |
| train/                |          |
|    entropy_loss       | -0.107   |
|    explained_variance | 0.84     |
|    learning_rate      | 0.0007   |
|    n_updates          | 34999    |
|    policy_loss        | 0.0121   |
|    value_loss         | 0.177    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 810      |
|    ep_rew_mean        | 14.5     |
| time/                 |          |
|    fps                | 175      |
|    iterations         | 35100    |
|    time_elapsed       | 4005     |
|    total_timesteps    | 702000   |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 814      |
|    ep_rew_mean        | 14       |
| time/                 |          |
|    fps                | 174      |
|    iterations         | 36300    |
|    time_elapsed       | 4163     |
|    total_timesteps    | 726000   |
| train/                |          |
|    entropy_loss       | -0.147   |
|    explained_variance | 0.833    |
|    learning_rate      | 0.0007   |
|    n_updates          | 36299    |
|    policy_loss        | -0.231   |
|    value_loss         | 0.131    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 830      |
|    ep_rew_mean        | 14.5     |
| time/                 |          |
|    fps                | 174      |
|    iterations         | 36400    |
|    time_elapsed       | 4173     |
|    total_timesteps    | 728000   |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 816      |
|    ep_rew_mean        | 14       |
| time/                 |          |
|    fps                | 174      |
|    iterations         | 37600    |
|    time_elapsed       | 4307     |
|    total_timesteps    | 752000   |
| train/                |          |
|    entropy_loss       | -0.15    |
|    explained_variance | 0.806    |
|    learning_rate      | 0.0007   |
|    n_updates          | 37599    |
|    policy_loss        | -0.0256  |
|    value_loss         | 0.126    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 822      |
|    ep_rew_mean        | 14.1     |
| time/                 |          |
|    fps                | 174      |
|    iterations         | 37700    |
|    time_elapsed       | 4322     |
|    total_timesteps    | 754000   |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 851      |
|    ep_rew_mean        | 14.8     |
| time/                 |          |
|    fps                | 174      |
|    iterations         | 38900    |
|    time_elapsed       | 4457     |
|    total_timesteps    | 778000   |
| train/                |          |
|    entropy_loss       | -0.243   |
|    explained_variance | 0.824    |
|    learning_rate      | 0.0007   |
|    n_updates          | 38899    |
|    policy_loss        | -0.0161  |
|    value_loss         | 0.131    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 865      |
|    ep_rew_mean        | 15.3     |
| time/                 |          |
|    fps                | 174      |
|    iterations         | 39000    |
|    time_elapsed       | 4467     |
|    total_timesteps    | 780000   |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 852      |
|    ep_rew_mean        | 14.9     |
| time/                 |          |
|    fps                | 172      |
|    iterations         | 40200    |
|    time_elapsed       | 4650     |
|    total_timesteps    | 804000   |
| train/                |          |
|    entropy_loss       | -0.144   |
|    explained_variance | 0.919    |
|    learning_rate      | 0.0007   |
|    n_updates          | 40199    |
|    policy_loss        | -0.117   |
|    value_loss         | 0.0952   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 842      |
|    ep_rew_mean        | 14.6     |
| time/                 |          |
|    fps                | 172      |
|    iterations         | 40300    |
|    time_elapsed       | 4665     |
|    total_timesteps    | 806000   |
| train/                |          |
|

## 7. Save or Reload trained model

In [10]:
save_trained_models_path = os.path.join('breakout_training', 'trained_models', 'A2C_model')

In [11]:
model.save(save_trained_models_path)

### Reload

In [ ]:
del model

In [14]:
env = make_atari_env('Breakout-v0', n_envs=1, seed=0)
env = VecFrameStack(env, n_stack=4)

In [ ]:
model = A2C.load(save_trained_models_path, env)

## 8. Evaluate model performance

In [18]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

(21.0, 7.224956747275377)

### Explore logs

In [12]:
!tensorboard --logdir={log_path}


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.7.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


### Results (for 1M steps)

<img src="./res/breakout_1mil_eval.png" width="500px" height="700px" />
<img src="./res/breakout_1mil_rollout.png" width="500px" height="700px" />

Reward target of **20** was reached.  
Evaluate policy returns reward of **15-21** with ~7 std.dev.

## 9. Test env with trained model

In [ ]:
observations = env.reset()
while True:
    action, states = model.predict(obs)
    observations, rewards, is_done, info = env.step(action)
    env.render()
env.close()

## 10. Interesting facts

Training a model for **1 million timesteps** took about **1,5 hours**.  

Laptop specs:
<img src="./res/laptop_specs.png" width="500px" height="700px" />